In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum

spark = SparkSession.builder.getOrCreate()

# Use Unity Catalog
CATALOG = "workspace"
SCHEMA = "fca_regulatory"

# Load validated transactions
txns = spark.table(f"{CATALOG}.{SCHEMA}.silver_validated_transactions")

# ---- CAPITAL ADEQUACY RATIO ----
loans = txns.filter(col("transaction_type") == "Loan") \
    .withColumn("rwa", col("amount") * 0.5)

total_rwa = loans.agg(spark_sum("rwa")).collect()[0][0]
capital = 1_000_000
car = round(capital / total_rwa, 2)

spark.createDataFrame([(car,)], ["capital_adequacy_ratio"]) \
    .write.format("delta").mode("overwrite") \
    .saveAsTable(f"{CATALOG}.{SCHEMA}.gold_capital_adequacy_ratio")

print(f"✅ Capital Adequacy Ratio: {car}")

# ---- LIQUIDITY COVERAGE RATIO ----
deposits = txns.filter(col("transaction_type") == "Deposit") \
    .agg(spark_sum("amount")).collect()[0][0]

withdrawals = txns.filter(col("transaction_type") == "Withdrawal") \
    .agg(spark_sum("amount")).collect()[0][0]

lcr = round(deposits / withdrawals, 2)

spark.createDataFrame([(lcr,)], ["liquidity_coverage_ratio"]) \
    .write.format("delta").mode("overwrite") \
    .saveAsTable(f"{CATALOG}.{SCHEMA}.gold_liquidity_coverage_ratio")

print(f"✅ Liquidity Coverage Ratio: {lcr}")

# ---- AML - SUSPICIOUS TRANSACTIONS (> £10k) ----
aml = txns.filter(col("amount") > 10000)

aml.write.format("delta").mode("overwrite") \
    .saveAsTable(f"{CATALOG}.{SCHEMA}.gold_suspicious_transactions")

print(f"✅ Suspicious transactions flagged: {aml.count()}")

# ---- CONDUCT RISK (Vulnerable Customers) ----
cust = spark.table(f"{CATALOG}.{SCHEMA}.silver_customer_360")

conduct = txns.join(
    cust.filter(col("customer_status") == "Vulnerable"), 
    "customer_id"
).filter(col("transaction_type") == "Loan")

conduct.write.format("delta").mode("overwrite") \
    .saveAsTable(f"{CATALOG}.{SCHEMA}.gold_conduct_risk_alerts")

print(f"✅ Conduct risk alerts: {conduct.count()}")

# ---- VERIFY ALL GOLD TABLES ----
print("\n📊 Gold Layer Summary:")
print(f"Capital Adequacy Ratio: {car}")
print(f"Liquidity Coverage Ratio: {lcr}")

spark.sql(f"""
    SELECT 'Suspicious Transactions' as metric, COUNT(*) as count 
    FROM {CATALOG}.{SCHEMA}.gold_suspicious_transactions
    UNION ALL
    SELECT 'Conduct Risk Alerts' as metric, COUNT(*) as count 
    FROM {CATALOG}.{SCHEMA}.gold_conduct_risk_alerts
""").show()

✅ Capital Adequacy Ratio: 0.61
✅ Liquidity Coverage Ratio: 0.97
✅ Suspicious transactions flagged: 533
✅ Conduct risk alerts: 125

📊 Gold Layer Summary:
Capital Adequacy Ratio: 0.61
Liquidity Coverage Ratio: 0.97
+--------------------+-----+
|              metric|count|
+--------------------+-----+
|Suspicious Transa...|  533|
| Conduct Risk Alerts|  125|
+--------------------+-----+

